In [3]:
%pip install -q pandas pyarrow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import json
from pathlib import Path
import pandas as pd

BASE = Path.cwd().resolve()
# Se você abriu o notebook na raiz, ajuste para BASE / "data"
DATA_DIR = (BASE / "data") if (BASE / "data").exists() else (BASE.parent / "data")

def load_json_flex(path: Path) -> pd.DataFrame:
    # 1) NDJSON
    try:
        df = pd.read_json(path, lines=True)
        if not df.empty:
            return df
    except Exception:
        pass

    # 2) array/dict
    try:
        with path.open("r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return pd.json_normalize(data, max_level=1)
        if isinstance(data, dict):
            for k, v in data.items():
                if isinstance(v, list):
                    return pd.json_normalize(v, max_level=1)
            return pd.json_normalize(data, max_level=1)
    except Exception:
        pass

    # 3) linhas JSON
    with path.open("r", encoding="utf-8") as f:
        rows = [json.loads(line) for line in f if line.strip()]
    return pd.json_normalize(rows, max_level=1)


In [5]:
vagas_path = DATA_DIR / "raw" / "vagas.json" if (DATA_DIR / "raw" / "vagas.json").exists() else DATA_DIR / "vagas.json"
apps_path  = DATA_DIR / "raw" / "applicants.json" if (DATA_DIR / "raw" / "applicants.json").exists() else DATA_DIR / "applicants.json"
pros_path  = DATA_DIR / "raw" / "prospects.json" if (DATA_DIR / "raw" / "prospects.json").exists() else DATA_DIR / "prospects.json"

df_vagas = load_json_flex(vagas_path)
df_apps  = load_json_flex(apps_path)
df_pros  = load_json_flex(pros_path)

print("Shapes:")
print("vagas    :", df_vagas.shape)
print("applicants:", df_apps.shape)
print("prospects :", df_pros.shape)

df_vagas.head(3), df_apps.head(3), df_pros.head(3)


Shapes:
vagas    : (1, 42243)
applicants: (1, 297374)
prospects : (1, 42666)


(                            5185.informacoes_basicas  \
 0  {'data_requicisao': '04-05-2021', 'limite_espe...   
 
                                     5185.perfil_vaga  \
 0  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
 
                                      5185.beneficios  \
 0  {'valor_venda': '-', 'valor_compra_1': 'R$', '...   
 
                             5184.informacoes_basicas  \
 0  {'data_requicisao': '04-05-2021', 'limite_espe...   
 
                                     5184.perfil_vaga  \
 0  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
 
                                      5184.beneficios  \
 0  {'valor_venda': '-', 'valor_compra_1': 'R$', '...   
 
                             5183.informacoes_basicas  \
 0  {'data_requicisao': '04-05-2021', 'limite_espe...   
 
                                     5183.perfil_vaga  \
 0  {'pais': 'Brasil', 'estado': 'São Paulo', 'cid...   
 
                                      5183.beneficios  \
 0  {'valor_ven

In [9]:
# Célula 6 — Normalizar dados "largos" (colunas = ID.campo) para formato tabular

import pandas as pd
from pandas import json_normalize

def _ensure_series(df):
    if len(df) != 1:
        raise ValueError("Esperava 1 linha; recebi {}. Verifique o loader.".format(len(df)))
    return df.iloc[0]

def normalize_vagas(df_vagas_raw: pd.DataFrame) -> pd.DataFrame:
    """Converte colunas como '5185.informacoes_basicas', '5185.perfil_vaga', ... em linhas por job_id."""
    row = _ensure_series(df_vagas_raw)
    buckets = {}

    for col, val in row.items():
        if not isinstance(col, str) or "." not in col:
            # ignora colunas sem o padrão <id>.<campo>
            continue
        job_id, field = col.split(".", 1)
        rec = buckets.setdefault(job_id, {"job_id": job_id})

        # Se o valor for dict, achata mantendo prefixo do campo
        if isinstance(val, dict):
            flat = json_normalize(val, sep=".").iloc[0].to_dict()
            flat = {f"{field}.{k}": v for k, v in flat.items()}
            rec.update(flat)
        else:
            rec[field] = val

    df = pd.DataFrame.from_records(list(buckets.values()))
    # texto unificado da vaga (título/senioridade/descrição/requisitos/desejáveis, se existirem)
    def build_text(r):
        parts = []
        for c in [
            "informacoes_basicas.titulo_vaga",
            "informacoes_basicas.senioridade",
            "perfil_vaga.descricao",
            "perfil_vaga.requisitos",
            "perfil_vaga.desejaveis",
        ]:
            v = r.get(c)
            if isinstance(v, str) and v.strip():
                parts.append(" ".join(v.split()))
        return " | ".join(parts)
    df["vaga_text"] = df.apply(build_text, axis=1)
    return df

def normalize_prospects(df_pros_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Procura colunas '<job_id>.prospects' que contêm listas de candidatos.
    Explode em linhas: job_id, codigo (candidato), situacao_candidado, etc.
    """
    row = _ensure_series(df_pros_raw)
    rows = []
    for col, val in row.items():
        if not isinstance(col, str) or not col.endswith(".prospects"):
            continue
        job_id = col.split(".", 1)[0]
        if isinstance(val, list):
            for item in val:
                if isinstance(item, dict):
                    rec = {"job_id": job_id}
                    rec.update(item)
                    rows.append(rec)
    if not rows:
        return pd.DataFrame(columns=["job_id", "codigo", "situacao_candidado"])
    df = pd.DataFrame(rows)
    # padroniza chaves úteis
    if "codigo" in df.columns:
        df["applicant_id"] = df["codigo"].astype(str)
    elif "codigo_profissional" in df.columns:
        df["applicant_id"] = df["codigo_profissional"].astype(str)
    else:
        # tenta achar alguma coluna de id de candidato
        cand_col = next((c for c in df.columns if "codigo" in c or "id" in c), None)
        if cand_col:
            df["applicant_id"] = df[cand_col].astype(str)
    # status
    status_col = next(
        (c for c in ["situacao_candidado","situacao_candidato","status","etapa"] if c in df.columns),
        None
    )
    if status_col is None:
        df["status"] = None
        status_col = "status"
    else:
        df = df.rename(columns={status_col: "status"})
    # tipifica
    df["job_id"] = df["job_id"].astype(str)
    df["applicant_id"] = df["applicant_id"].astype(str)
    return df[["job_id","applicant_id","status"] + [c for c in df.columns if c not in ["job_id","applicant_id","status"]]]

def normalize_applicants(df_apps_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Converte colunas '<applicant_id>.<campo>' para linhas por candidato.
    Tenta extrair cv_pt e agrega demais campos.
    """
    row = _ensure_series(df_apps_raw)
    buckets = {}

    for col, val in row.items():
        if not isinstance(col, str) or "." not in col:
            continue
        app_id, field = col.split(".", 1)
        rec = buckets.setdefault(app_id, {"applicant_id": app_id})

        if isinstance(val, dict):
            flat = json_normalize(val, sep=".").iloc[0].to_dict()
            flat = {f"{field}.{k}": v for k, v in flat.items()}
            rec.update(flat)
        else:
            rec[field] = val

    df = pd.DataFrame.from_records(list(buckets.values()))

    # Escolhe um campo de CV principal
    def pick_cv(r):
        for c in ["cv_pt", "cv", "curriculo", "resumo"]:
            v = r.get(c)
            if isinstance(v, str) and v.strip():
                return " ".join(v.split())
        # tenta campos textuais que pareçam descrição/experiência
        texts = []
        for c, v in r.items():
            if isinstance(v, str) and any(k in c.lower() for k in ["resumo","experiencia","descricao","cv"]):
                v2 = v.strip()
                if v2:
                    texts.append(" ".join(v2.split()))
        return " | ".join(texts) if texts else None

    df["cv_text"] = df.apply(pick_cv, axis=1)
    return df

# ---- executar normalização
df_vagas_norm = normalize_vagas(df_vagas)
df_pros_norm  = normalize_prospects(df_pros)
df_apps_norm  = normalize_applicants(df_apps)

print("vagas_norm:", df_vagas_norm.shape)
print("prospects_norm:", df_pros_norm.shape)
print("applicants_norm:", df_apps_norm.shape)

display(df_vagas_norm.head(3))
display(df_pros_norm.head(3))
display(df_apps_norm[["applicant_id","cv_text"]].head(3))


vagas_norm: (14081, 46)
prospects_norm: (53759, 9)
applicants_norm: (42482, 59)


,job_id,informacoes_basicas.data_requicisao,informacoes_basicas.limite_esperado_para_contratacao,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.cliente,informacoes_basicas.solicitante_cliente,informacoes_basicas.empresa_divisao,informacoes_basicas.requisitante,informacoes_basicas.analista_responsavel,...,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,beneficios.valor_venda,beneficios.valor_compra_1,beneficios.valor_compra_2,informacoes_basicas.data_inicial,informacoes_basicas.data_final,perfil_vaga.habilidades_comportamentais_necessarias,informacoes_basicas.nome_substituto,vaga_text
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN,Operation Lead -
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN,Consultor PP/QM Sênior
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN,ANALISTA PL/JR C/ SQL


,job_id,applicant_id,status,nome,codigo,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,25632,Encaminhado ao Requisitante,José Vieira,25632,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,25529,Encaminhado ao Requisitante,Srta. Isabela Cavalcante,25529,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,25364,Contratado pela Decision,Sra. Yasmin Fernandes,25364,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano


,applicant_id,cv_text
0,31000,assistente administrativo santosbatista itapec...
1,31001,formação acadêmica ensino médio (2º grau) em e...
2,31002,objetivo: área administrativa | financeira res...


In [10]:
# Célula 7 — Gerar pares (vaga ↔ candidato) rotulados e salvar em disco

from pathlib import Path
import pandas as pd

# 1) mapeamento de status → label
LABEL_POS = {
    "Aprovado",
    "Contratado pela Decision",
    "Contratado como Hunting",
    # Inclua se quiser considerar como positivo:
    # "Documentação PJ",
}
LABEL_NEG = {
    "Não Aprovado pelo Cliente",
    "Não Aprovado pelo RH",
    "Não Aprovado pelo Requisitante",
    "Não Aprovado",
    "Desistiu",
    "Sem interesse nesta vaga",
}

def map_label(s):
    if not isinstance(s, str):
        return None
    s = s.strip()
    if s in LABEL_POS: return 1
    if s in LABEL_NEG: return 0
    return None  # neutro (vamos descartar do treino)

# 2) aplica rótulo e filtra neutros
df_pros_labeled = df_pros_norm.copy()
df_pros_labeled["label"] = df_pros_labeled["status"].map(map_label)
df_pros_labeled = df_pros_labeled.dropna(subset=["label"]).copy()
df_pros_labeled["label"] = df_pros_labeled["label"].astype(int)

# 3) seleciona textos principais
vagas_text = df_vagas_norm[["job_id", "vaga_text"]].drop_duplicates()
apps_text  = df_apps_norm[["applicant_id", "cv_text"]].drop_duplicates()

# 4) junta (job_id, applicant_id, status, label) → vaga_text, cv_text
pairs = (
    df_pros_labeled[["job_id","applicant_id","status","label"]]
    .merge(vagas_text, on="job_id", how="left")
    .merge(apps_text, on="applicant_id", how="left")
)

# 5) limpeza final: remover pares sem texto
pairs = pairs.fillna({"vaga_text": "", "cv_text": ""})
pairs = pairs[(pairs["vaga_text"].str.len() > 0) & (pairs["cv_text"].str.len() > 0)].copy()

# 6) salvar
out_dir = Path("data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
pairs_path = out_dir / "pairs.parquet"
pairs_sample_path = out_dir / "pairs_sample.csv"

pairs.to_parquet(pairs_path, index=False)
pairs.head(200).to_csv(pairs_sample_path, index=False, encoding="utf-8")

# 7) sanity check
n_total = len(pairs)
n_pos = int((pairs["label"]==1).sum())
n_neg = int((pairs["label"]==0).sum())
print("Pares tot.:", n_total, "| +:", n_pos, "| -:", n_neg)
print("Arquivos salvos em:")
print(" -", pairs_path)
print(" -", pairs_sample_path)

pairs.head(5)


Pares tot.: 9778 | +: 2475 | -: 7303
Arquivos salvos em:
 - data\processed\pairs.parquet
 - data\processed\pairs_sample.csv


,job_id,applicant_id,status,label,vaga_text,cv_text
0,4531,25364,Contratado pela Decision,1,2021-2607395-PeopleSoft Application Engine-Dom...,área de atuação: lider de consultoria / gerenc...
1,4533,26338,Contratado pela Decision,1,2021-2605708-Microfocus Application Life Cycle...,"solteiro, brasileiro, 21/06/1987 habilitação c..."
2,4533,24645,Desistiu,0,2021-2605708-Microfocus Application Life Cycle...,analista de teste/qa profissional hands on sem...
3,4534,26205,Desistiu,0,2021-2605711-Microfocus QTP - UFT Automation T...,"idade: 37 anos 172 apto.2703, vila maria josé ..."
4,4534,26003,Não Aprovado pelo Cliente,0,2021-2605711-Microfocus QTP - UFT Automation T...,solteira – 40 anos – brasileira itaquaquecetub...
